In [1]:
%matplotlib inline
import pandas as pd
import os
import numpy as np
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import sklearn
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
NEWLINE = '\n'
SKIP_FILES = {'cmds'}


def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
        for file_name in file_names:
            if file_name not in SKIP_FILES:
                file_path = os.path.join(root, file_name)
                if os.path.isfile(file_path):
                    past_header, lines = False, []
                    f = open(file_path, encoding="ANSI")
                    for line in f:
                        if past_header:
                            lines.append(line)
                        elif line == NEWLINE:
                            past_header = True
                    f.close()
                    content = NEWLINE.join(lines)
                    yield content
                    


In [3]:
path =r'C:\Users\osutr_000\Documents\Data\Ops'
list_ = []

for value in read_files(path):
    list_.append(value)

ops_df = pd.DataFrame(data = list_)
ops_df['class']="ops"
len(ops_df)

580

In [4]:
path =r'C:\Users\osutr_000\Documents\Data\Legal'
list_ = []

for value in read_files(path):
    list_.append(value)

legal_df = pd.DataFrame(data = list_)
legal_df['class']="legal"
len(legal_df)

209

In [5]:
path =r'C:\Users\osutr_000\Documents\Data\Accounting'
list_ = []

for value in read_files(path):
    list_.append(value)

act_df = pd.DataFrame(data = list_)
act_df['class']="accounting"
len(act_df)

324

In [6]:
merged_df = ops_df.append(legal_df).append(act_df)
len(merged_df)
merged_df.columns = ['text', 'cat']
merged_df.sort_index().reset_index().head()

,index,text,cat
0,0,"401 W. 33rd \n\nEdmond, OK 73013 \n\nWWW.MIDCO...",ops
1,0,26938 \n\n\n\n002 \n\n\n\n4/27/16 290.60 290. ...,accounting
2,0,"MIDCON Data Services, LLC. \n\n\n\nThis Data D...",legal
3,1,5/25/16 179.17 179.17 \n\n\n\n\n\n5/27/16 0160...,accounting
4,1,,legal


In [10]:
def make_xy(merged_df, vectorizer=None):
    #Your code here    
    if vectorizer is None:
        vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(merged_df.text)
    X = X.tocsc()  # some versions of sklearn return COO format
    y = (merged_df.cat == 'ops').values.astype(np.int)
    return X, y
X, y = make_xy(merged_df)


In [15]:
y.sum()

580

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
xtrain, xtest, ytrain, ytest = train_test_split(X, y)
classi = MultinomialNB().fit(xtrain, ytrain)
accuracy_test = classi.score(xtest, ytest)
accuracy_train = classi.score(xtrain, ytrain)
print("Accuracy of test set")
print(accuracy_test)
print("Accuracy of training set")
print(accuracy_train)

Accuracy of test set
0.924731182796
Accuracy of training set
0.928057553957
